In [ ]:
import numpy as np
import operator
from math import sqrt
import sklearn
from sklearn import datasets
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import warnings
from sklearn.exceptions import ConvergenceWarning
"""
https://arxiv.org/pdf/1805.08125.pdf

Allocation Function. AF : (Pn, bn) → Sn, Sn ⊂ [M], the allocation function, takes as input the
price vector Pn and the bid from a buyer bn, to decide which features the buyer gets allocated.

Revenue Function. RF : (Pn, bn, Yn ;M, G) → rn, rn ∈ R+, the revenue function, takes as
input the current price vector Pn, the bid and the prediction task provided by the buyer (bn and Yn
respectively), to decide how much revenue rn to extract from the buyer.

Payment Division Function. PD : (Sn, Yn ;M, G) → ψn, ψn ∈ [0, 1]|Sn|
, the payment-division function, takes as input the set of features that were allocated XSn
along with the prediction task Yn,
to compute ψn, a vector denoting the marginal value of each allocated feature for the prediction task.

Price Update Function. PF : (Pn, bn, Yn) → Pn+1, Pn+1 ∈ RM
+ , the price-update function, takes
as input the current price vector Pn, the bid and the prediction task provided by the buyer (bn and Yn
respectively) to update the price vector for each of the M features.
"""

"""
Property A.1: Algorithm converges to the maximum price vector over time
Description of Experiment 1.1
Take n example datasets from Scikit-Learn [(X_i), Y_i]
[(X_1), (X_2), …, (X_n)] are features on sale
[Y_1, Y_2, …., Y_n] are prediction tasks “for hire”
[b_1, b_2, ….., b_n] are bids for each prediction type  
Dynamics
Uniformly at random select (Y_n, b_n) as stream of buyers
Desired outcomes of experiment  
(P_i) converge to approximately b_i 
Why? Optimal Outcome is that price of each (feature set type) converges to bid for that that prediction task type
Regularization helps
Why? - Theory shows that it makes the problem “convex”
"""
dset_names = [
 'load_boston',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
]
    
dsets = {}
for attr in dset_names:
    name = attr.split('load_')[1]
    if name:
        dsets[name] = getattr(datasets, attr)()
Y_dict = { name: dsets[name]['target'] for name in dsets }
X_dict = { name: dsets[name]['data'] for name in dsets }

In [ ]:
[(key, x.shape) for key, x in X_dict.items()]

In [ ]:
[(key, y.shape) for key, y in Y_dict.items()]

In [ ]:
""" Look at first four, up to 400 time steps"""
X_t1 = np.concatenate([x[:400] for x in X_dict.values()], axis=1)
Y_t1 = np.stack([y[:400] for y in Y_dict.values()])

In [ ]:
X_t1.shape, Y_t1.shape

In [ ]:

def get_original_columns(X_n, feature_idxs):
    indices = [idx for (idx, price) in feature_idxs]
    return X_n[:, indices]

def gain_score(y, x):
    r2 = r2_score(y, x)
    return r2 if r2 > 0 else 0

In [ ]:
# Allocation and Revenue function
# Section 4.1
def revenue_func(P_n, b_n, Y_n, context, random_state):
    """
    P_n: the price vector at previous timestep (t=n)
    b_n: the bid at previous timestep (in $ / model_score)
    Y_n: the test Y data set
    """
    X_n = context.get('X_n')
    gain_func = context.get('gain_func')
    if X_n is None or gain_func is None:
        return None, None
    
    feature_count = len(P_n)
    feature_options = sorted(list((idx, p) for (idx, p) in enumerate(P_n) if p <= b_n), key=operator.itemgetter(1))
    print('feature_options: {}'.format(feature_options))
    if not feature_options:
        return 0, None
    
    assert feature_count == X_n.shape[1], 'feature_count: {}, X_n.shape[1]: {}'.format(feature_count, X_n.shape[1])

    # 90% train and cross-validate, 10% test
    print('X_n shape: {}'.format(X_n.shape))
    print('Y_n shape: {}'.format(Y_n.shape))
    X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, Y_n, test_size=0.1, random_state=random_state)

    # Gain results by slice index
    feature_indices = []
    gain_results = []
    for (idx, (feature_idx, feature_price)) in enumerate(feature_options):
        _train = get_original_columns(X_n_train, feature_idxs=feature_options[:idx+1])
        _test = get_original_columns(X_n_test, feature_idxs=feature_options[:idx+1])
        gain_func.fit(_train, y_n_train)
        result = gain_score(y_n_test, gain_func.predict(_test))
        previous_result = gain_results[-1] if gain_results else 0
        feature_indices.append(feature_idx)
        # Only add a feature if it improves predictive gain
        # This enforces the necessary property of the gain function
        # always being monotonic in the features
        if (result > previous_result):
            gain_results.append(result)
        else:
            gain_results.append(previous_result)

    if not gain_results:
        raise Exception("gain_results empty")

    # Add the bid as the rightmost price
    feature_indices += [-1]
    P_n += [b_n]
    revenue_list = []
    prev_gain = 0
    for idx, gain in enumerate(gain_results):
        price_idx = feature_indices[idx]
        price = P_n[price_idx]
        gain = gain_results[idx]
        gain_difference = gain - prev_gain
        assert gain_difference >= 0, 'gain {}, prev_gain {}'.format(gain, prev_gain)
        prev_gain = gain
        revenue = gain_difference * price
        revenue_list.append(revenue)
    return sum(revenue_list), {
        'revenue_list': revenue_list, 
        'gain_results': gain_results, 
        'feature_indices': feature_indices
    }


In [ ]:
X_n = X_t1
X_n.shape

In [ ]:
# Test 1: price of all features equal to bid (all allocated, revenue = best gain * bid)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
def test1(random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n,
    }
    feature_count = X_n.shape[1]
    b_n = 5
    P_n = [b_n] * feature_count
    
    # TODO: test each instead of randomly
    y_choice_idx = np.random.choice(Y_t1.shape[0])
    Y_n = Y_t1[y_choice_idx]

    revenue, _ = revenue_func(P_n, b_n, Y_n, context, random_state)

    X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, Y_n, test_size=0.1, random_state=random_state)
    clf.fit(X_n_train, y_n_train)
    expected_revenue = gain_score(y_n_test, clf.predict(X_n_test)) * b_n
    print(expected_revenue)
#     assert np.isclose(revenue, expected_revenue), "{} != {}".format(revenue, expected_revenue)
    assert revenue >= expected_revenue, "{} != {}".format(revenue, expected_revenue)
test1()

In [ ]:
def test2(random_state=100):
    # Test that no features are allocated (revenue zero) when all price are greater than the bid
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n,
    }
    feature_count = X_n.shape[1]
    b_n = 0.05
    P_n = [0.1] * feature_count
    y_choice_idx = np.random.choice(Y_t1.shape[0])
    Y_n = Y_t1[y_choice_idx]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue, _ = revenue_func(P_n, b_n, Y_n, context, random_state)
    assert revenue == 0
test2()

In [ ]:
def test3(feature_count=3, random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n[:, :feature_count],
    }
    b_n = 5
    P_n = [6, 3, 2]
    Y_n = Y_t1[3]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue, _ = revenue_func(P_n, b_n, Y_n, context, random_state)
    print(revenue)
    print(_)
    P_n = [1, 1, 6]
    revenue, _ = revenue_func(P_n, b_n, Y_n, context, random_state)
    print(revenue)
test3()

In [ ]:
# Payment Division Function

In [ ]:
def payment_division_func(feature_indices, Y_n, M, gain_func, K):
    """
    S_n: the features allocated to buyer n
    Y_n: the test Y data set
    M: the number of sellers
    gain_func: the gain function
    K: the number of times to sample uniformly from S_n
    """
    for m in feature_indicesn:
        for k in range(K):
            sigma_k = np.random.permutation(feature_indices)
            #TODO: continue from line 4 of Algorithm 2
#             _train = get_original_columns(X_n_train, feature_idxs=feature_options[:idx+1])
#             _test = get_original_columns(X_n_test, feature_idxs=feature_options[:idx+1])
#             gain_func.fit(_train, y_n_train)
#             result = gain_score(y_n_test, gain_func.predict(_test))
#             gain = gain_func(Y_n, )

In [ ]:
# Price Update Function

In [ ]:
# @Anish - look into whether this is useful, for now, leave it out
# def smooth_kink(price_vec, bid, c):
#     capped_price = bid + 1 / (2 * c)
#     def _smooth(price):
#         return price - (c * (price - bid) ** 2) / 2
#     return [
#         p if p <= bid else
#         _smooth(p) if p <= bid + 1 / c else 
#         capped_price 
#         for p in price_vec
#     ]


def price_update_func(price_vec, bid, Y, c):
#     smooth_prices = smooth_kink(price_vec, bid, c)
    
    clf = linear_model.LassoCV(cv=5, random_state=100)
    context = {
        'gain_func': clf,
        'X_n': X_n[:, :3],
    }
    revenue, output = revenue_func(price_vec, bid, Y, context, random_state=100)
    revenue_list = output['revenue_list']
    revenue_list_gradient = [
        r - revenue_list[idx - 1]
        if idx > 0 else r
        for idx, r, in enumerate(revenue_list)
    ]
    print(revenue_list_gradient)

price_update_func([2, 3, 4], 3, Y_t1[3], c=10)
